# Model Iteration 1 (Linear Regression)
To start, we are going to implement a linear regression set for the data. Then we will iterate, and improve, the model.

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv("train.csv")

First things first, we are going to clean days of week to be values ranging from 0 - 6 (Mon - Sun) using a dictionary and mapping.

In [89]:
dow = {
    "Monday" : 0,
    "Tuesday" : 1,
    "Wednesday" : 2,
    "Thursday" : 3,
    "Friday" : 4,
    "Saturday" : 5,
    "Sunday" : 6
}
data["DOW"] = data.DayOfWeek.map(dow)

Next step is similar, but now with Police Districts so we can just deal with ints instead of strings.

In [102]:
pds = {
    "SOUTHERN" : 0,
    "MISSION" : 1,
    "NORTHERN" : 2,
    "BAYVIEW" : 3,
    "CENTRAL" : 4,
    "TERNDERLOIN" : 5,
    "INGLESIDE" : 6,
    "TARAVAL" : 7,
    "PARK" : 8,
    "RICHMOND" : 9
}
data["pds"] = data.PdDistrict.map(pds)
# for crimes without PD, use "Other" : 10
data["pds"] = data["pds"].fillna(10)

Next, we are going to use X and Y coordinates as they are pretty continuous. There are some outlying data points, so let's clean that up. First, we check for those outlying data points (which we happen to know are at the north pole).

In [103]:
for val in data.Y:
    if val == 90.0:
        print val
        
for val in data.X:
    if val == -120.5:
        print val

Then we replace those points for more reasonable numbers. Running the code below, then the code above will produce 0 outlying values (because we got rid of them).

In [104]:
data.X.replace(-120.5, data["X"].median(), inplace = True)
data.Y.replace(90, data["Y"].median(), inplace = True)

The data cleaning train has no brakes. Now we are going to clean up date/time.

In [105]:
def parse_date(Dates):
    """ Convert a date in YYYY-MM-DD HH:MM:SS to a tuple
        containing year, month, day, and hours each expressed
        as an integer. Used from Paul Ruvolo's example in bikeshare kaggle dataset

        >>> parse_date("2014-04-05 14:00:00")
        (2014, 4, 5, 14)
    """
    return int(Dates[0:4]), int(Dates[5:7]), int(Dates[8:10]), int(Dates[11:13])

# "now we use the lambda functions" - Mack
data["Year"] = data.Dates.apply(lambda x: parse_date(x)[0])
data["Month"] = data.Dates.apply(lambda x: parse_date(x)[1])
data["Hour"] = data.Dates.apply(lambda x: parse_date(x)[3])

And so now we are going to do the predictive thingamajig. We drew inspiration from this script (https://www.kaggle.com/sonuk7/sf-crime/prediction-with-bernoulinb/code)

In [115]:
from sklearn.naive_bayes import BernoulliNB
from sklearn import cross_validation

cats = data.Category.values
cleanData = data.drop(["Address","Category","Dates","Descript","X", "Y","Resolution", "DayOfWeek", "PdDistrict"], axis=1)

model = BernoulliNB()
model.fit(cleanData.dropna(), cats)

scores = cross_validation.cross_val_score(model, cleanData, data["Category"], cv = 3)
print scores

[ 0.19976153  0.19864289  0.19928998]


Wow that's a bad score :\